# AtmoML Batch Predict

Runs AtmoML prediction on Google Cloud Batch.

In [5]:
from google.cloud import batch_v1
import time


from usl_models.shared import batch_templates

timestamp = time.strftime("%Y%m%d-%H%M%S")

In [4]:
# Build a container using the current directory.
build_container = False
if build_container:
    !gcloud builds submit --config=usl_models/batch_predict/atmo_ml/cloudbuild.yaml .

e103032c49c9: Pushed
dev: digest: sha256:87ca6c55521c6ce62ab2c7033305ebf98fcdc3de8bd4004a3d802ad6caf5f1f5 size: 3901
DONE
--------------------------------------------------------------------------------
ID                                    CREATE_TIME                DURATION  SOURCE                                                                                   IMAGES                                                            STATUS
a6bb9bc7-9574-4168-ad79-4e927d2c0268  2025-03-27T01:46:59+00:00  12M40S    gs://climateiq_cloudbuild/source/1743040018.686553-23b85e5a844841f997b5924691720361.tgz  us-central1-docker.pkg.dev/climateiq/usl-models/atmo-ml-test:dev  SUCCESS


In [ ]:
# Run batch job
client = batch_v1.BatchServiceClient()

container = batch_v1.Runnable.Container(
    image_uri=batch_templates.CLIMATEIQ_IMAGE_URI,
    entrypoint="python3.11",
    commands=[
        "/usl_models/batch_predict/atmo_ml/main.py",
        "--model_path=gs://climateiq-vertexai/atmoml-main-20250319-204950/model",
        "--batch_size=2",
        "--max_batches=4",
        "--output_bucket=climateiq-predictions",
        "--output_path=atmoml-test/atmoml-main-20250319-204950",
    ],
)
job = batch_v1.Job(
    allocation_policy=batch_v1.AllocationPolicy(
        instances=[batch_templates.NVIDIA_L4_X2_INSTANCE]
    ),
    logs_policy=batch_v1.LogsPolicy(
        destination=batch_v1.LogsPolicy.Destination.CLOUD_LOGGING
    ),
    task_groups=[
        batch_v1.TaskGroup(
            require_hosts_file=True,
            task_count=2,
            task_count_per_node=1,
            task_spec=batch_v1.TaskSpec(
                runnables=[
                    batch_v1.Runnable(container=container),
                ],
                environment=batch_v1.Environment(variables={}),
            ),
        )
    ],
)
request = batch_v1.CreateJobRequest(
    job_id=f"atmoml-predict-{timestamp}",
    job=job,
    parent="projects/climateiq/locations/us-central1",
)

response = client.create_job(request)
response

In [ ]:
batch_templates.await_batch_job(client, job_id=request.job_id, job_name=response.name)